In [1]:
%env PYART_QUIET=true

env: PYART_QUIET=true


In [4]:
cd ../data/01

/Users/tuphamminh/Documents/GitHub/MF-Airflow/data/01


In [2]:
import os
import glob
import numpy as np
import xarray as xr
import netCDF4 as nc
from tqdm import tqdm
from matplotlib import pyplot as plt
import wradlib as wrl
from matplotlib.widgets import Cursor 
import matplotlib
import datetime
import cartopy
import pandas as pd

In [3]:
MAP_PROJECTION = cartopy.crs.PlateCarree()

In [5]:
import pyart

# Đường dẫn đến tập tin dữ liệu radar
# file_path = 'NHB230602000007.RAWLHXD'
file_path = '2023-06-01T06:00'

# Đọc tập tin radar
radar = pyart.io.read(file_path)

# # Hiển thị thông tin cơ bản về radar
print(radar.info())

altitude:
	data: <ndarray of type: float64 and shape: (1,)>
	long_name: Altitude
	standard_name: Altitude
	units: meters
	positive: up
altitude_agl: None
antenna_transition: None
azimuth:
	data: <ndarray of type: float32 and shape: (1388,)>
	units: degrees
	standard_name: beam_azimuth_angle
	long_name: azimuth_angle_from_true_north
	axis: radial_azimuth_coordinate
	comment: Azimuth of antenna relative to true north
elevation:
	data: <ndarray of type: float32 and shape: (1388,)>
	units: degrees
	standard_name: beam_elevation_angle
	long_name: elevation_angle_from_horizontal_plane
	axis: radial_elevation_coordinate
	comment: Elevation of antenna relative to the horizontal plane
fields:
	total_power:
		data: <ndarray of type: float32 and shape: (1388, 500)>
		units: dBZ
		standard_name: equivalent_reflectivity_factor
		long_name: Total power
		coordinates: elevation azimuth range
		_FillValue: -9999.0
	reflectivity:
		data: <ndarray of type: float32 and shape: (1388, 500)>
		units: dBZ
		

In [ ]:
from matplotlib.axes import Axes
from matplotlib.figure import Figure


processed_grid = pyart.map.grid_from_radars(
    radar,
    grid_shape=(10, 640, 640),
    grid_limits=((0, 100), (-300_000, 300_000), (-300_000, 300_000)),
)

# # Extract reflectivity data from the grid
# reflectivity_data = processed_grid.fields['reflectivity']['data'][:]

# # Find maximum reflectivity at each grid point across all PPI sweeps
# cmax_data = np.max(reflectivity_data, axis=0)

# # for i in range(0,100):
# #     processed_grid.fields['reflectivity']['data'][i] = cmax_data
# processed_grid.fields['reflectivity']['data'][0:100] = cmax_data

fig: Figure
ax: Axes
fig = plt.figure(dpi=1000)

ax = plt.axes(projection=MAP_PROJECTION)

ax.add_feature(
    cartopy.feature.NaturalEarthFeature(
        category="physical", name="coastline", scale="10m", facecolor="none"
    ),
    linestyle="-",
    edgecolor="k",
    linewidth=1,
)

grid_display = pyart.graph.GridMapDisplay(processed_grid)
grid_display.plot_grid(
    field="reflectivity",
    level=3,
    cmap="pyart_HomeyerRainbow",
    fig=fig,
    ax=ax,
    projection=MAP_PROJECTION,
    embellish=False,
    lon_lines=np.linspace(
        np.round(processed_grid.to_xarray().lon.min(), 2),
        np.round(processed_grid.to_xarray().lon.max(), 2),
        5,
    ),
    lat_lines=np.linspace(
        np.round(processed_grid.to_xarray().lat.min(), 2),
        np.round(processed_grid.to_xarray().lat.max(), 2),
        5,
    ),
)

# fig.savefig(ppi)
